
The general recipe is a short list of four main steps:

1.   Compose a function to **read** input data and prepare a Tensorflow Dataset;
2.   Define a **scoring** function that, given a (set of) query-document feature vector(s), produces a score indicating the query's level of relevance to the document;
3.   Create a **loss** function that measures how far off the produced scores from step (2) are from the ground truth; and,
4.   Define evaluation **metrics**.

## Imports

In [24]:
import tensorflow as tf
import tensorflow_ranking as tfr
import pandas as pd

tf.enable_eager_execution()
tf.executing_eagerly()

True

## Constants

In [60]:
# Store the paths to files containing training and test instances.
# As noted above, we will assume the data is in the LibSVM format
# and that the content of each file is sorted by query ID.

_TRAIN_DATA_PATH="../data/interim/train_pid.libsvm"
_TEST_DATA_PATH="../data/interim/test_p.libsvm"

# Define a loss function. To find a complete list of available
# loss functions or to learn how to add your own custom function
# please refer to the tensorflow_ranking.losses module.
_LOSS="pairwise_logistic_loss"

# In the TF-Ranking framework, a training instance is represented
# by a Tensor that contains features from a list of documents
# associated with a single query. For simplicity, we fix the shape
# of these Tensors to a maximum list size and call it "list_size,"
# the maximum number of documents per query in the dataset.
# In this demo, we take the following approach:
#   * If a query has fewer documents, its Tensor will be padded
#     appropriately.
#   * If a query has more documents, we shuffle its list of
#     documents and trim the list down to the prescribed list_size.
_LIST_SIZE=100

# The total number of features per query-document pair.
# We set this number to the number of features in the MSLR-Web30K
# dataset.
_NUM_FEATURES=4

# Parameters to the scoring function.
_BATCH_SIZE=100
_HIDDEN_LAYER_DIMS=["20", "10"]

#_OUT_DIR = "../models/tfranking/"

## Function to read in data and form tensorflow dataset

Out train and test dataset is in the lib svm format which is normally used for Support Vector Machines. 

In [15]:
fo = open(_TRAIN_DATA_PATH)
i=0
for f in fo:
    if i != 10:
        print(f)
    else:
        break
    #i+=1

0 qid:10 1:3 2:47429 3:4604 4:700

0 qid:10 1:4 2:47429 3:4604 4:14100

0 qid:10 1:8 2:49758 3:6878 4:5500

0 qid:10 1:11 2:47796 3:7234 4:3000

1 qid:10 1:1 2:48995 3:7396 4:3200

0 qid:10 1:2 2:49067 3:6345 4:3100

0 qid:21 1:4 2:7667 3:1229 4:2100

0 qid:21 1:3 2:7667 3:929 4:700

1 qid:21 1:2 2:6157 3:1289 4:300

1 qid:25 1:1 2:3006 3:2419 4:200

0 qid:25 1:4 2:3898 3:669 4:1300

0 qid:25 1:6 2:3521 3:1063 4:700

1 qid:25 1:1 2:3014 3:2432 4:200

0 qid:25 1:5 2:2714 3:2434 4:700

0 qid:25 1:3 2:3898 3:429 4:700

0 qid:34 1:4 2:24642 3:3530 4:9200

0 qid:34 1:7 2:33507 3:5470 4:1000

0 qid:34 1:3 2:24642 3:3410 4:700

1 qid:34 1:9 2:26039 3:4411 4:600

0 qid:34 1:2 2:25925 3:5663 4:600

0 qid:35 1:3 2:3341 3:729 4:700

0 qid:35 1:4 2:3341 3:849 4:1300

1 qid:35 1:1 2:4415 3:2050 4:200

0 qid:35 1:6 2:3586 3:1084 4:700

1 qid:35 1:1 2:4008 3:2176 4:200

0 qid:35 1:5 2:3066 3:2736 4:700

1 qid:36 1:1 2:3453 3:1801 4:200

0 qid:36 1:3 2:2622 3:483 4:700

0 qid:36 1:5 2:2569 3:2310 4:70

0 qid:3788 1:3 2:30327 3:2210 4:700

0 qid:3788 1:4 2:30327 3:2330 4:8200

1 qid:3788 1:1 2:31985 3:6082 4:700

1 qid:3800 1:2 2:27490 3:4043 4:2800

0 qid:3800 1:3 2:26638 3:3348 4:700

0 qid:3803 1:5 2:4578 3:4071 4:700

0 qid:3803 1:6 2:5155 3:1558 4:700

0 qid:3803 1:4 2:5199 3:1023 4:1800

0 qid:3803 1:3 2:5199 3:783 4:700

1 qid:3803 1:7 2:5176 3:2171 4:500

0 qid:3803 1:1 2:6455 3:2212 4:200

0 qid:3809 1:3 2:13675 3:1326 4:700

1 qid:3809 1:1 2:10518 3:4730 4:200

0 qid:3809 1:6 2:7768 3:2350 4:700

1 qid:3809 1:1 2:15014 3:5161 4:500

0 qid:3809 1:4 2:13675 3:1626 4:3400

0 qid:3816 1:3 2:21987 3:4081 4:700

0 qid:3816 1:4 2:21987 3:4081 4:7300

1 qid:3816 1:2 2:21396 3:3194 4:500

1 qid:3822 1:2 2:18447 3:2679 4:500

0 qid:3822 1:3 2:20767 3:3531 4:700

0 qid:3822 1:4 2:20767 3:3831 4:5700

1 qid:3826 1:2 2:21804 3:3458 4:500

0 qid:3826 1:3 2:20731 3:2349 4:700

0 qid:3826 1:4 2:20731 3:2469 4:7100

0 qid:3826 1:10 2:22774 3:3428 4:2100

0 qid:3826 1:7 2:22067 3:4284 4:700



0 qid:7632 1:3 2:16715 3:2168 4:700

0 qid:7632 1:7 2:14527 3:4246 4:600

1 qid:7632 1:1 2:14652 3:4684 4:300

0 qid:7633 1:6 2:4810 3:1452 4:700

0 qid:7633 1:4 2:4789 3:2209 4:2500

0 qid:7633 1:5 2:4724 3:4335 4:700

1 qid:7633 1:2 2:8325 3:2212 4:400

0 qid:7633 1:3 2:4789 3:1909 4:700

0 qid:7633 1:1 2:4589 3:2516 4:200

0 qid:7639 1:9 2:6097 3:1911 4:300

0 qid:7639 1:4 2:5168 3:1199 4:1900

1 qid:7639 1:3 2:5168 3:899 4:700

0 qid:7639 1:6 2:4756 3:1431 4:700

0 qid:7639 1:5 2:4057 3:3541 4:700

0 qid:7639 1:1 2:5283 3:2367 4:200

0 qid:7639 1:2 2:6094 3:2247 4:300

0 qid:7641 1:6 2:836 3:252 4:700

0 qid:7641 1:7 2:3040 3:2215 4:1800

0 qid:7641 1:1 2:1752 3:1504 4:200

1 qid:7641 1:5 2:1052 3:901 4:700

0 qid:7641 1:4 2:1846 3:699 4:1300

0 qid:7641 1:3 2:1846 3:399 4:700

0 qid:7653 1:3 2:21484 3:3996 4:700

0 qid:7653 1:4 2:21484 3:4116 4:7400

1 qid:7653 1:2 2:18890 3:2368 4:500

0 qid:7657 1:4 2:18235 3:2005 4:6200

1 qid:7657 1:1 2:18797 3:5691 4:500

0 qid:7657 1:3 2:182

0 qid:11384 1:3 2:20884 3:2494 4:700

0 qid:11384 1:9 2:21628 3:3394 4:500

0 qid:11384 1:1 2:22482 3:5602 4:500

0 qid:11384 1:4 2:20884 3:2614 4:5200

1 qid:11384 1:7 2:22236 3:4143 4:700

0 qid:11385 1:6 2:1576 3:475 4:700

0 qid:11385 1:5 2:1104 3:984 4:700

0 qid:11385 1:4 2:2083 3:438 4:1300

0 qid:11385 1:3 2:2083 3:438 4:700

1 qid:11385 1:1 2:1732 3:1055 4:200

0 qid:11386 1:5 2:4318 3:3859 4:700

1 qid:11386 1:7 2:6445 3:2000 4:500

0 qid:11386 1:6 2:4449 3:1343 4:700

0 qid:11386 1:3 2:5419 3:992 4:700

0 qid:11386 1:4 2:5419 3:1232 4:1800

0 qid:11386 1:1 2:5455 3:2742 4:200

0 qid:11386 1:9 2:5816 3:1544 4:300

0 qid:11391 1:1 2:13764 3:4845 4:400

0 qid:11391 1:2 2:15744 3:2880 4:500

1 qid:11391 1:3 2:18365 3:1988 4:700

0 qid:11391 1:10 2:16209 3:2344 4:1800

0 qid:11391 1:9 2:15731 3:2460 4:500

0 qid:11391 1:4 2:18365 3:2108 4:4700

0 qid:11394 1:3 2:2159 3:413 4:700

0 qid:11394 1:6 2:2503 3:750 4:700

0 qid:11394 1:5 2:1996 3:1816 4:700

1 qid:11394 1:1 2:2864 3:158

1 qid:14641 1:7 2:11015 3:3725 4:500

0 qid:14641 1:3 2:12328 3:1961 4:700

0 qid:14641 1:1 2:10943 3:4013 4:300

0 qid:14641 1:6 2:8960 3:2712 4:700

0 qid:14650 1:3 2:2 3:14 4:700

1 qid:14650 1:5 2:88 3:75 4:700

0 qid:14650 1:6 2:83 3:24 4:700

0 qid:14663 1:3 2:11618 3:1993 4:700

0 qid:14663 1:4 2:11618 3:2293 4:3900

1 qid:14663 1:7 2:12874 3:3859 4:500

0 qid:14681 1:4 2:1974 3:744 4:1300

0 qid:14681 1:3 2:1974 3:444 4:700

0 qid:14681 1:1 2:1789 3:1563 4:200

0 qid:14681 1:6 2:1782 3:537 4:700

1 qid:14681 1:5 2:1460 3:1247 4:700

0 qid:14681 1:1 2:1993 3:1641 4:200

1 qid:14696 1:2 2:24830 3:5210 4:600

0 qid:14696 1:9 2:24830 3:4938 4:600

0 qid:14696 1:3 2:30147 3:5001 4:700

0 qid:14696 1:4 2:30147 3:5241 4:9300

0 qid:14696 1:7 2:26919 3:6892 4:700

0 qid:14698 1:3 2:28861 3:2192 4:700

0 qid:14698 1:2 2:38759 3:4485 4:700

0 qid:14698 1:4 2:28861 3:2432 4:9000

0 qid:14698 1:7 2:37077 3:4746 4:900

1 qid:14698 1:10 2:38529 3:3997 4:2100

0 qid:14709 1:3 2:1510 3:629 4:7

...


1 qid:69792 1:2 2:6994 3:1912 4:300

0 qid:69792 1:3 2:8428 3:1611 4:700

0 qid:69792 1:1 2:5991 3:2595 4:200

0 qid:69792 1:4 2:8428 3:1791 4:3100

0 qid:69793 1:1 2:19272 3:6307 4:500

1 qid:69793 1:9 2:14830 3:3031 4:500

0 qid:69793 1:3 2:16445 3:3200 4:700

0 qid:69793 1:4 2:16445 3:3440 4:4800

0 qid:69793 1:2 2:14828 3:3528 4:500

0 qid:69794 1:3 2:16493 3:3099 4:700

1 qid:69794 1:2 2:16491 3:3258 4:500

0 qid:69794 1:9 2:17207 3:3183 4:500

0 qid:69794 1:4 2:16493 3:3279 4:6300

0 qid:69794 1:7 2:16780 3:5036 4:600

1 qid:69797 1:1 2:19904 3:5193 4:500

1 qid:69797 1:1 2:20384 3:5583 4:500

0 qid:69797 1:4 2:14485 3:1269 4:2900

0 qid:69797 1:3 2:14485 3:1209 4:700

0 qid:69814 1:4 2:29903 3:4017 4:13500

1 qid:69814 1:7 2:42860 3:8464 4:1400

0 qid:69814 1:1 2:33991 3:10071 4:700

0 qid:69814 1:3 2:29903 3:3897 4:700

0 qid:69819 1:4 2:11147 3:3995 4:5700

1 qid:69819 1:2 2:11015 3:2147 4:400

0 qid:69819 1:3 2:11147 3:3875 4:700

0 qid:69827 1:9 2:17378 3:3391 4:500

0 qid:


1 qid:74179 1:2 2:7026 3:1850 4:400

0 qid:74179 1:3 2:7883 3:1407 4:700

0 qid:74179 1:4 2:7883 3:1587 4:3200

0 qid:74179 1:6 2:6442 3:1945 4:700

0 qid:74179 1:7 2:6568 3:2490 4:500

0 qid:74184 1:6 2:1406 3:422 4:700

0 qid:74184 1:4 2:1393 3:536 4:1300

0 qid:74184 1:5 2:1401 3:1220 4:700

0 qid:74184 1:3 2:1393 3:356 4:700

1 qid:74184 1:1 2:2009 3:1390 4:200

1 qid:74186 1:5 2:884 3:776 4:700

0 qid:74186 1:3 2:917 3:135 4:700

0 qid:74186 1:6 2:1004 3:303 4:700

0 qid:74186 1:1 2:1485 3:1170 4:200

0 qid:74196 1:4 2:16071 3:2415 4:4500

0 qid:74196 1:11 2:22550 3:5507 4:700

0 qid:74196 1:7 2:22258 3:5630 4:900

1 qid:74196 1:1 2:15966 3:4517 4:2200

0 qid:74196 1:3 2:16071 3:2235 4:700

0 qid:74208 1:2 2:44661 3:7397 4:700

0 qid:74208 1:4 2:34246 3:3794 4:11300

1 qid:74208 1:3 2:34246 3:3614 4:700

0 qid:74208 1:7 2:44757 3:8048 4:900

0 qid:74209 1:1 2:44750 3:9401 4:1000

0 qid:74209 1:4 2:34216 3:3918 4:11400

0 qid:74209 1:3 2:34216 3:3678 4:700

1 qid:74209 1:2 2:44619

1 qid:78752 1:2 2:9983 3:2471 4:400

0 qid:78752 1:3 2:9495 3:2435 4:700

0 qid:78752 1:4 2:9495 3:2615 4:3400

1 qid:78760 1:2 2:6173 3:1263 4:300

0 qid:78760 1:3 2:8606 3:891 4:700

0 qid:78760 1:6 2:6081 3:1836 4:700

0 qid:78760 1:4 2:8606 3:1131 4:2000

0 qid:78761 1:4 2:8804 3:1323 4:2000

0 qid:78761 1:3 2:8804 3:1023 4:700

0 qid:78761 1:6 2:6362 3:1921 4:700

1 qid:78761 1:2 2:6274 3:1350 4:300

0 qid:78782 1:4 2:1720 3:486 4:1300

0 qid:78782 1:1 2:1839 3:1273 4:200

0 qid:78782 1:6 2:1718 3:519 4:700

1 qid:78782 1:5 2:1714 3:1528 4:700

0 qid:78782 1:2 2:3567 3:1869 4:300

0 qid:78782 1:3 2:1720 3:306 4:700

1 qid:78787 1:4 2:2596 3:724 4:1400

0 qid:78787 1:5 2:2502 3:2313 4:700

0 qid:78787 1:1 2:5034 3:2250 4:200

0 qid:78787 1:3 2:2596 3:544 4:700

0 qid:78787 1:1 2:3731 3:2181 4:200

0 qid:78787 1:6 2:2522 3:761 4:700

1 qid:78788 1:1 2:7755 3:2623 4:200

0 qid:78788 1:3 2:4584 3:635 4:700

0 qid:78788 1:4 2:4584 3:875 4:2300

0 qid:78788 1:5 2:4380 3:4000 4:700

1 qi

0 qid:82865 1:10 2:16343 3:3193 4:2800

0 qid:82865 1:4 2:13973 3:2104 4:4700

0 qid:82865 1:3 2:13973 3:1864 4:700

1 qid:82865 1:2 2:12893 3:2598 4:400

0 qid:82874 1:10 2:14076 3:2578 4:1900

1 qid:82874 1:2 2:13015 3:2831 4:400

0 qid:82874 1:3 2:15464 3:3848 4:700

0 qid:82874 1:4 2:15464 3:4148 4:5500

1 qid:82879 1:2 2:14728 3:3097 4:500

0 qid:82879 1:7 2:13551 3:3441 4:600

0 qid:82879 1:4 2:12023 3:2060 4:4000

1 qid:82881 1:7 2:26918 3:7194 4:900

0 qid:82881 1:1 2:27180 3:8917 4:800

0 qid:82881 1:4 2:25922 3:2723 4:10100

0 qid:82881 1:3 2:25922 3:2543 4:700

0 qid:82884 1:7 2:3136 3:1972 4:500

0 qid:82884 1:4 2:2970 3:1550 4:2300

0 qid:82884 1:6 2:2942 3:888 4:700

1 qid:82884 1:2 2:3383 3:1326 4:300

0 qid:82884 1:5 2:2661 3:2356 4:700

0 qid:82884 1:3 2:2970 3:1310 4:700

0 qid:82888 1:4 2:8139 3:890 4:1300

0 qid:82888 1:1 2:1503 3:1033 4:200

0 qid:82888 1:3 2:8139 3:710 4:700

0 qid:82888 1:6 2:1133 3:343 4:700

1 qid:82888 1:5 2:1133 3:1000 4:700

0 qid:82888 1:1 

KeyboardInterrupt: 

In this example they first number shows the target

1 = Important 

0 = Not important

qid: Describes which lines belong together

E.g. query 10 had 6 suggestions for plans and just one is important. Here we would take the suggested transport mode from this plan. 

### Input Pipeline

The first step to construct an input pipeline that reads your dataset and produces a `tensorflow.data.Dataset` object. In this example, we will invoke a LibSVM parser that is included in the `tensorflow_ranking.data` module to generate a `Dataset` from a given file.

We parameterize this function by a `path` argument so that the function can be used to read both training and test data files.

 Read tf DataSet
 
 Dic for feature

In [6]:
def input_fn(path):
    train_dataset = tf.data.Dataset.from_generator(
      tfr.data.libsvm_generator(path, _NUM_FEATURES, _LIST_SIZE),
      output_types=(
          {str(k): tf.float32 for k in range(1,_NUM_FEATURES+1)},
          tf.float32
      ),
      output_shapes=(
          {str(k): tf.TensorShape([_LIST_SIZE, 1])
            for k in range(1,_NUM_FEATURES+1)},
          tf.TensorShape([_LIST_SIZE])
      )
    )

    train_dataset = train_dataset.shuffle(1000).repeat().batch(_BATCH_SIZE)
    return train_dataset.make_one_shot_iterator().get_next()

In [7]:
train_dataset = tf.data.Dataset.from_generator(
      tfr.data.libsvm_generator(_TRAIN_DATA_PATH, _NUM_FEATURES, _LIST_SIZE),
      output_types=(
          {str(k): tf.float32 for k in range(1,_NUM_FEATURES+1)},
          tf.float32
      ),
      output_shapes=(
          {str(k): tf.TensorShape([_LIST_SIZE, 1])
            for k in range(1,_NUM_FEATURES+1)},
          tf.TensorShape([_LIST_SIZE])
      )
    )

train_dataset

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


<DatasetV1Adapter shapes: ({1: (10, 1), 2: (10, 1), 3: (10, 1), 4: (10, 1)}, (10,)), types: ({1: tf.float32, 2: tf.float32, 3: tf.float32, 4: tf.float32}, tf.float32)>

In [8]:
a=train_dataset.make_one_shot_iterator().get_next()

Instructions for updating:
Colocations handled automatically by placer.


In [14]:
a[0]

{'1': <tf.Tensor: id=28, shape=(10, 1), dtype=float32, numpy=
 array([[ 4.],
        [ 8.],
        [ 1.],
        [ 3.],
        [11.],
        [ 2.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.]], dtype=float32)>,
 '2': <tf.Tensor: id=29, shape=(10, 1), dtype=float32, numpy=
 array([[47429.],
        [49758.],
        [48995.],
        [47429.],
        [47796.],
        [49067.],
        [    0.],
        [    0.],
        [    0.],
        [    0.]], dtype=float32)>,
 '3': <tf.Tensor: id=30, shape=(10, 1), dtype=float32, numpy=
 array([[4604.],
        [6878.],
        [7396.],
        [4604.],
        [7234.],
        [6345.],
        [   0.],
        [   0.],
        [   0.],
        [   0.]], dtype=float32)>,
 '4': <tf.Tensor: id=31, shape=(10, 1), dtype=float32, numpy=
 array([[14100.],
        [ 5500.],
        [ 3200.],
        [  700.],
        [ 3000.],
        [ 3100.],
        [    0.],
        [    0.],
        [    0.],
        [    0.]], dtype=float32)>}

## Estimator Creation


Next, we turn to the scoring function which is arguably at the heart of a TF Ranking model. The idea is to compute a relevance score for a (set of) query-document pair(s). The TF-Ranking model will use training data to learn this function.

Here we formulate a scoring function using a feed forward network. The function takes the features of a single example (i.e., query-document pair) and produces a relevance score

In [16]:
def example_feature_columns():
    """Returns the example feature columns."""
    feature_names = [
      "%d" % (i + 1) for i in range(0, _NUM_FEATURES)
    ]
    return {
      name: tf.feature_column.numeric_column(
          name, shape=(1,), default_value=0.0) for name in feature_names
    }

def make_score_fn(mode):
        """Returns a scoring function to build `EstimatorSpec`."""
        
        def _score_fn(context_features, group_features, mode, params, config):
            """Defines the network to score a documents."""
            del params
            del config
            # Define input layer.
            example_input = [
                tf.layers.flatten(group_features[name])
                for name in sorted(example_feature_columns())
            ]
            print("MAKE SCORE FUNCTION:")
            print(example_input)
            input_layer = tf.concat(example_input, 1)

            cur_layer = input_layer
            for i, layer_width in enumerate(int(d) for d in _HIDDEN_LAYER_DIMS):
                cur_layer = tf.layers.dense(
                  cur_layer,
                  units=layer_width,
                  activation="tanh")

            logits = tf.layers.dense(cur_layer, units=1)
            return logits
        return _score_fn

In [17]:
def eval_metric_fns():
    """Returns a dict from name to metric functions.

    This can be customized as follows. Care must be taken when handling padded
    lists.

    def _auc(labels, predictions, features):
    is_label_valid = tf_reshape(tf.greater_equal(labels, 0.), [-1, 1])
    clean_labels = tf.boolean_mask(tf.reshape(labels, [-1, 1], is_label_valid)
    clean_pred = tf.boolean_maks(tf.reshape(predictions, [-1, 1], is_label_valid)
    return tf.metrics.auc(clean_labels, tf.sigmoid(clean_pred), ...)
    metric_fns["auc"] = _auc

    Returns:
    A dict mapping from metric name to a metric function with above signature.
    """
    metric_fns = {}
    metric_fns.update({
      "metric/ndcg@%d" % topn: tfr.metrics.make_ranking_metric_fn(
          tfr.metrics.RankingMetricKey.NDCG, topn=topn)
      for topn in [1, 3, 5, 10]
    })

    return metric_fns

In [18]:
def get_estimator(hparams, mode):
    """Create a ranking estimator.

    Args:
    hparams: (tf.contrib.training.HParams) a hyperparameters object.

    Returns:
    tf.learn `Estimator`.
    """
    def _train_op_fn(loss):
        """Defines train op used in ranking head."""
        return tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.train.get_global_step(),
            learning_rate=hparams.learning_rate,
            optimizer="Adagrad")

    ranking_head = tfr.head.create_ranking_head(
      loss_fn=tfr.losses.make_loss_fn(_LOSS),
      eval_metric_fns=eval_metric_fns(),
      train_op_fn=_train_op_fn)

    return tf.estimator.Estimator(
      model_fn=tfr.model.make_groupwise_ranking_fn(
          group_score_fn=make_score_fn(mode),
          group_size=1,
          transform_fn=None,
          ranking_head=ranking_head),
        params=hparams)

In [168]:
hparams = tf.contrib.training.HParams(learning_rate=0.05)
ranker = get_estimator(hparams, tf.estimator.ModeKeys.TRAIN)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp6prtmlao', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f68c3990a58>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
ranker.train(input_fn=lambda: input_fn(_TRAIN_DATA_PATH), steps=100000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
MAKE SCORE FUNCTION:
[<tf.Tensor 'groupwise_dnn_v2/group_score/flatten/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_1/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_2/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_3/Reshape:0' shape=(?, 1) dtype=float32>]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.


/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp6prtmlao/model.ckpt.
INFO:tensorflow:loss = 0.74308383, step = 1
INFO:tensorflow:global_step/sec: 16.9774
INFO:tensorflow:loss = 0.5465798, step = 101 (5.892 sec)
INFO:tensorflow:global_step/sec: 17.5112
INFO:tensorflow:loss = 0.63619304, step = 201 (5.711 sec)
INFO:tensorflow:global_step/sec: 17.213
INFO:tensorflow:loss = 0.46078545, step = 2401 (5.810 sec)
INFO:tensorflow:global_step/sec: 17.3458
INFO:tensorflow:loss = 0.52249134, step = 2501 (5.765 sec)
INFO:tensorflow:global_step/sec: 17.1133
INFO:tensorflow:loss = 0.5488479, step = 2601 (5.844 sec)
INFO:tensorflow:global_step/sec: 17.3848
INFO:tensorflow:loss = 0.6157759, step = 2701 (5.751 sec)
INFO:tensorflow:global_step/sec: 17.6194
INFO:tensorflow:loss = 0.5658778, step = 2801 (5.676 sec)
INFO:tensorflow:global_step/sec: 17.6502
INFO:tensorflow:loss = 0.52557725, step = 2901 (5.665 sec)
INFO:

INFO:tensorflow:global_step/sec: 17.7198
INFO:tensorflow:loss = 0.5171158, step = 10301 (5.644 sec)
INFO:tensorflow:global_step/sec: 17.4266
INFO:tensorflow:loss = 0.5190744, step = 10401 (5.738 sec)
INFO:tensorflow:Saving checkpoints for 10458 into /tmp/tmp6prtmlao/model.ckpt.
INFO:tensorflow:global_step/sec: 16.9943
INFO:tensorflow:loss = 0.50714266, step = 10501 (5.884 sec)
INFO:tensorflow:global_step/sec: 17.5508
INFO:tensorflow:loss = 0.51337093, step = 10601 (5.698 sec)
INFO:tensorflow:global_step/sec: 17.3986
INFO:tensorflow:loss = 0.42842263, step = 10701 (5.749 sec)
INFO:tensorflow:global_step/sec: 17.2183
INFO:tensorflow:loss = 0.41773838, step = 10801 (5.806 sec)
INFO:tensorflow:global_step/sec: 17.3931
INFO:tensorflow:loss = 0.46903872, step = 10901 (5.749 sec)
INFO:tensorflow:global_step/sec: 17.0727
INFO:tensorflow:loss = 0.54238987, step = 11001 (5.858 sec)
INFO:tensorflow:global_step/sec: 17.4019
INFO:tensorflow:loss = 0.50693655, step = 11101 (5.746 sec)
INFO:tensorflo

In [170]:
ranker

In [24]:
test_X

,sid,click_time,click_mode,distance_plan,eta,price,transport_mode,plan_time,pid,req_time,o_long,o_lat,d_long,d_lat,distance_query,target
4,2629085,2018-10-12 16:28:13,3,13207,2790,400.0,9,2018-10-12 16:27:55,203797.0,2018-10-12 16:27:55,116.35,40.08,116.33,40.03,5.808139,0
5,2629085,2018-10-12 16:28:13,3,8175,1656,700.0,3,2018-10-12 16:27:55,203797.0,2018-10-12 16:27:55,116.35,40.08,116.33,40.03,5.808139,1
6,2629085,2018-10-12 16:28:13,3,8175,1776,2700.0,4,2018-10-12 16:27:55,203797.0,2018-10-12 16:27:55,116.35,40.08,116.33,40.03,5.808139,0
7,2629085,2018-10-12 16:28:13,3,9620,2424,1600.0,10,2018-10-12 16:27:55,203797.0,2018-10-12 16:27:55,116.35,40.08,116.33,40.03,5.808139,0
8,2629085,2018-10-12 16:28:13,3,8079,2443,700.0,6,2018-10-12 16:27:55,203797.0,2018-10-12 16:27:55,116.35,40.08,116.33,40.03,5.808139,0
9,2629085,2018-10-12 16:28:13,3,13428,3777,800.0,7,2018-10-12 16:27:55,203797.0,2018-10-12 16:27:55,116.35,40.08,116.33,40.03,5.808139,0
0,2848914,2018-11-17 18:42:17,1,53156,6456,700.0,1,2018-11-17 12:56:15,101804.0,2018-11-17 12:56:15,116.36,40.07,116.00,40.35,43.656570,1
1,2848914,2018-11-17 18:42:17,1,48112,3535,700.0,3,2018-11-17 12:56:15,101804.0,2018-11-17 12:56:15,116.36,40.07,116.00,40.35,43.656570,0
2,2848914,2018-11-17 18:42:17,1,48112,3655,16500.0,4,2018-11-17 12:56:15,101804.0,2018-11-17 12:56:15,116.36,40.07,116.00,40.35,43.656570,0
3,2848914,2018-11-17 18:42:17,1,51641,8871,1200.0,1,2018-11-17 12:56:15,101804.0,2018-11-17 12:56:15,116.36,40.07,116.00,40.35,43.656570,1


In [25]:
fo = open('../data/interim/test_10.libsvm')

for i in fo:
    print(i)

0 qid:2629085 1:9 2:13207 3:2790 4:400

1 qid:2629085 1:3 2:8175 3:1656 4:700

0 qid:2629085 1:4 2:8175 3:1776 4:2700

0 qid:2629085 1:10 2:9620 3:2424 4:1600

0 qid:2629085 1:6 2:8079 3:2443 4:700

0 qid:2629085 1:7 2:13428 3:3777 4:800

1 qid:2848914 1:1 2:53156 3:6456 4:700

0 qid:2848914 1:3 2:48112 3:3535 4:700

0 qid:2848914 1:4 2:48112 3:3655 4:16500

1 qid:2848914 1:1 2:51641 3:8871 4:1200



In [26]:
X = pd.read_pickle("../data/interim/X_10.pickle")

In [27]:
X

,transport_mode,distance_plan,eta,price
4,9,13207,2790,400.0
5,3,8175,1656,700.0
6,4,8175,1776,2700.0
7,10,9620,2424,1600.0
8,6,8079,2443,700.0
9,7,13428,3777,800.0
0,1,53156,6456,700.0
1,3,48112,3535,700.0
2,4,48112,3655,16500.0
3,1,51641,8871,1200.0


In [28]:
test_X[[
    'sid',
    'transport_mode',
    'distance_plan',
    'eta', 
    'price'
]]

,sid,transport_mode,distance_plan,eta,price
4,2629085,9,13207,2790,400.0
5,2629085,3,8175,1656,700.0
6,2629085,4,8175,1776,2700.0
7,2629085,10,9620,2424,1600.0
8,2629085,6,8079,2443,700.0
9,2629085,7,13428,3777,800.0
0,2848914,1,53156,6456,700.0
1,2848914,3,48112,3535,700.0
2,2848914,4,48112,3655,16500.0
3,2848914,1,51641,8871,1200.0


In [29]:
next(preds)

array([4.226252  , 4.226252  , 4.226252  , 0.44031858, 0.01470059],
      dtype=float32)

In [171]:
preds = ranker.predict(input_fn=lambda: input_fn(_TEST_DATA_PATH), yield_single_examples=True)

In [78]:
type(preds)

generator

In [172]:
next(preds)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
MAKE SCORE FUNCTION:
[<tf.Tensor 'groupwise_dnn_v2/group_score/flatten/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_1/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_2/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_3/Reshape:0' shape=(?, 1) dtype=float32>]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp6prtmlao/model.ckpt-100000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


array([-0.94607913, -0.16126832, -2.3476968 ,  0.18963477,  0.18963477,
        0.18963477,  0.18963477,  0.18963477,  0.18963477,  0.18963477,
        0.18963477,  0.18963477,  0.18963477,  0.18963477,  0.18963477,
        0.18963477,  0.18963477,  0.18963477,  0.18963477,  0.18963477,
        0.18963477,  0.18963477,  0.18963477,  0.18963477,  0.18963477,
        0.18963477,  0.18963477,  0.18963477,  0.18963477,  0.18963477,
        0.18963477,  0.18963477,  0.18963477,  0.18963477,  0.18963477,
        0.18963477,  0.18963477,  0.18963477,  0.18963477,  0.18963477,
        0.18963477,  0.18963477,  0.18963477,  0.18963477,  0.18963477,
        0.18963477,  0.18963477,  0.18963477,  0.18963477,  0.18963477,
        0.18963477,  0.18963477,  0.18963477,  0.18963477,  0.18963477,
        0.18963477,  0.18963477,  0.18963477,  0.18963477,  0.18963477,
        0.18963477,  0.18963477,  0.18963477,  0.18963477,  0.18963477,
        0.18963477,  0.18963477,  0.18963477,  0.18963477,  0.18

In [123]:
test = df_train[:10]

In [127]:
import numpy as np
preds = ranker.predict(input_fn=lambda: input_fn(_TRAIN_DATA_PATH), yield_single_examples=True)
import itertools
preds_slice = itertools.islice(preds, _LIST_SIZE) # grab 
count=0
a = np.zeros((len(df_train), _LIST_SIZE))

for i in preds_slice:
    a[count]=i
    count+=1

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
MAKE SCORE FUNCTION:
[<tf.Tensor 'groupwise_dnn_v2/group_score/flatten/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_1/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_2/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_3/Reshape:0' shape=(?, 1) dtype=float32>]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp2f1tkotx/model.ckpt-11000


Exception ignored in: <generator object EstimatorV2.predict at 0x7f68c3864a98>
Traceback (most recent call last):
  File "/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/tensorflow_estimator/python/estimator/estimator.py", line 639, in predict
    for key, value in six.iteritems(preds_evaluated)
  File "/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/contextlib.py", line 130, in __exit__
    self.gen.throw(type, value, traceback)
  File "/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/tensorflow/python/framework/ops.py", line 5253, in get_controller
    yield g
  File "/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/contextlib.py", line 130, in __exit__
    self.gen.throw(type, value, traceback)
  File "/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/tensorflow/python/framework/ops.py", line 5069, in get_controller
    type(default))
AssertionError: Nesting violated for default stack of <class 'tensorflow.python.framewo

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [173]:
df_train  = pd.read_pickle("../data/processed/df_train.pickle")

In [174]:
df_train.head()

,sid,click_time,click_mode,distance_plan,eta,price,transport_mode,plan_time,pid,req_time,o_long,o_lat,d_long,d_lat,distance_query
0,2848914,2018-11-17 18:42:17,1,53156,6456,700.0,1,2018-11-17 12:56:15,101804.0,2018-11-17 12:56:15,116.36,40.07,116.00,40.35,43.656570
1,2848914,2018-11-17 18:42:17,1,48112,3535,700.0,3,2018-11-17 12:56:15,101804.0,2018-11-17 12:56:15,116.36,40.07,116.00,40.35,43.656570
2,2848914,2018-11-17 18:42:17,1,48112,3655,16500.0,4,2018-11-17 12:56:15,101804.0,2018-11-17 12:56:15,116.36,40.07,116.00,40.35,43.656570
3,2848914,2018-11-17 18:42:17,1,51641,8871,1200.0,1,2018-11-17 12:56:15,101804.0,2018-11-17 12:56:15,116.36,40.07,116.00,40.35,43.656570
4,2629085,2018-10-12 16:28:13,3,13207,2790,400.0,9,2018-10-12 16:27:55,203797.0,2018-10-12 16:27:55,116.35,40.08,116.33,40.03,5.808139


In [129]:
df_test = pd.read_pickle("../data/processed/df_test.pickle")

In [130]:
len(df_test)

429803

In [163]:
def ltr_to_submission(df, ranker):
    import numpy as np
    preds = ranker.predict(input_fn=lambda: input_fn(_TEST_DATA_PATH))
    import itertools
    preds_slice = itertools.islice(preds, _LIST_SIZE) # grab 
    count=0
    a = np.zeros((len(df), _LIST_SIZE))

    for i in preds_slice:
        a[count]=i
        count+=1
        
        
    test_X = df[[
       'sid',
        'transport_mode',
        'distance_plan',
        'eta', 
        'price'
    ]]
    
    # Assign prediction vals to df
    test_X = test_X.assign(yhat = a[:,0])
    
    df_end = pd.DataFrame(columns=['yhat'], index=df.sid.unique())

    # for sid in df.sid.unique():
    #    tmp = test_X.loc[test_X.sid == sid]
    #    val = tmp.loc[tmp['yhat'].idxmax()]['transport_mode']
    #    df_end.loc[sid, 'yhat'] = val
    
    msk = test_X.groupby('sid', sort=False)['yhat'].transform(max) == test_X.yhat
    df_end = test_X[msk][['sid', 'transport_mode']]
    
    df_end.sort_index(inplace=True)
    
    return df_end

In [164]:
df_train.shape

(2101060, 15)

In [189]:
import numpy as np
preds = ranker.predict(input_fn=lambda: input_fn(_TEST_DATA_PATH))
import itertools
preds_slice = itertools.islice(preds, _LIST_SIZE) # grab 
count=0
a = np.zeros((len(df_test), _LIST_SIZE))

for i in preds_slice:
    a[count]=i
    count+=1


test_X = df_test[[
   'sid',
    'transport_mode',
    'distance_plan',
    'eta', 
    'price'
]]

# Assign prediction vals to df
test_X = test_X.assign(yhat = a[:,0])

df_end = pd.DataFrame(columns=['yhat'], index=df_test.sid.unique())


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
MAKE SCORE FUNCTION:
[<tf.Tensor 'groupwise_dnn_v2/group_score/flatten/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_1/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_2/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_3/Reshape:0' shape=(?, 1) dtype=float32>]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp6prtmlao/model.ckpt-100000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [197]:
test_X.yhat.max()

0.9088973999023438

In [222]:
msk = test_X.groupby('sid', sort=False)['yhat'].transform(max) == test_X.yhat

In [224]:
df_end = test_X[msk][['sid', 'transport_mode']]

In [225]:
df_end.head()

,sid,transport_mode
1,1112456,6
3,1413458,2
8,1243160,2
15,2040494,10
19,1448779,4


In [175]:
df_preds = ltr_to_submission(df_train, ranker)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
MAKE SCORE FUNCTION:
[<tf.Tensor 'groupwise_dnn_v2/group_score/flatten/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_1/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_2/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_3/Reshape:0' shape=(?, 1) dtype=float32>]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp6prtmlao/model.ckpt-100000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Exception ignored in: <generator object EstimatorV2.predict at 0x7f68c1bd57c8>
Traceback (most recent call last):
  File "/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/tensorflow_estimator/python/estimator/estimator.py", line 639, in predict
    for key, value in six.iteritems(preds_evaluated)
  File "/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/contextlib.py", line 130, in __exit__
    self.gen.throw(type, value, traceback)
  File "/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/tensorflow/python/framework/ops.py", line 5253, in get_controller
    yield g
  File "/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/contextlib.py", line 130, in __exit__
    self.gen.throw(type, value, traceback)
  File "/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/tensorflow/python/framework/ops.py", line 5069, in get_controller
    type(default))
AssertionError: Nesting violated for default stack of <class 'tensorflow.python.framewo

In [180]:
df_preds.head()

,yhat
2848914,1
2629085,4
602598,5
2022975,7
988425,10


In [177]:
df_train.groupby('sid').first()["click_mode"].head(10)

sid
10    1
21    2
25    1
34    9
35    1
36    1
44    2
68    2
69    1
79    2
Name: click_mode, dtype: int64

In [184]:
df_preds.sort_index(inplace=True)

In [185]:
df_preds.head()

,yhat
10,2
21,2
25,1
34,9
35,1


In [226]:
f1_score(df_test.groupby("sid").first()['click_mode'], df_end.yhat, average='weighted')

KeyError: 'click_mode'